In [1]:
from collections import namedtuple

In [2]:
test = namedtuple('test', ['name', 'agent'])

In [ ]:
class Agent:
    def __init__(self, messages):
        self.messages = messages
    
    def add_message(self, message):
        self.messages.append(message)

In [ ]:
a = Agent([1])

In [ ]:
b = test('test1', a)

In [ ]:
b.agent.messages

In [ ]:
a.add_message(2)

In [ ]:
b.agent.messages

In [ ]:
value = 1
x = f"{{{value}}}"

In [ ]:
getattr('tools.navigator','get_available_destinations')

In [ ]:
x = {"activeStep": "Projects~tb_settings", "tool_name": "navigator", "task_finished": True}